In [3]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

# Load data
books = pd.read_csv('Books.csv.zip')
users = pd.read_csv('Users.csv.zip')
ratings = pd.read_csv('Ratings.csv.zip')


C:\Users\pruth\AppData\Local\Temp\ipykernel_18840\3172148919.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv.zip')


In [4]:
# Merge ratings with book details
ratings_with_name = ratings.merge(books, on='ISBN')
ratings_with_name

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...
1031131,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...
1031132,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...
1031133,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...
1031134,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...


In [5]:
# Filter users who have rated more than 50 books
user_ratings_count = ratings_with_name.groupby('User-ID').count()['Book-Rating']
active_users = user_ratings_count[user_ratings_count > 50].index
filtered_ratings = ratings_with_name[ratings_with_name['User-ID'].isin(active_users)]
# Filter books that have been rated at least 10 times
book_ratings_count = filtered_ratings.groupby('Book-Title').count()['Book-Rating']
popular_books = book_ratings_count[book_ratings_count >= 10].index
final_ratings = filtered_ratings[filtered_ratings['Book-Title'].isin(popular_books)]
final_ratings

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
5,23768,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
7,28523,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
10,56157,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...
1031125,276688,0060164220,0,Coyote Waits,Tony Hillerman,1990,Harpercollins,http://images.amazon.com/images/P/0060164220.0...,http://images.amazon.com/images/P/0060164220.0...,http://images.amazon.com/images/P/0060164220.0...
1031126,276688,0060168307,0,Sacred Clowns,Tony Hillerman,1994,Harpercollins,http://images.amazon.com/images/P/0060168307.0...,http://images.amazon.com/images/P/0060168307.0...,http://images.amazon.com/images/P/0060168307.0...
1031128,276688,0385471092,0,Mrs. Pollifax and the Second Thief,Dorothy Gilman,1993,Doubleday Books,http://images.amazon.com/images/P/0385471092.0...,http://images.amazon.com/images/P/0385471092.0...,http://images.amazon.com/images/P/0385471092.0...
1031131,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...


In [6]:
# Create a pivot table of ratings
pivot_table = final_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')
pivot_table.fillna(0, inplace=True)
pivot_table

User-ID,243,254,507,638,643,741,882,929,1211,1424,...,277928,277965,278026,278137,278144,278188,278418,278582,278633,278843
Book-Title,,,,,,,,,,,,,,,,,,,,,
Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback)),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Q-Space (Star Trek The Next Generation, Book 47)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01-01-00: The Novel of the Millennium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"1,000 Places to See Before You Die",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"\O\"" Is for Outlaw""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Create a utility matrix from final_ratings pivot table

pt_matrix = csr_matrix(pivot_table.values)
print(pt_matrix.toarray())

# Convert the pivot table into a sparse matrix
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(pt_matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


NearestNeighbors(algorithm='brute', metric='cosine')

In [9]:

# Recommendation function with suggestions for similar book names
def recommend(book):
    try:
        query_index = pivot_table.index.get_loc(book)
        distances, indices = model_knn.kneighbors(pt_matrix[query_index], n_neighbors=6)

        print(f"Recommendations for '{pivot_table.index[query_index]}':")
        for i in range(1, len(distances.flatten())):
            print(f"{i}: {pivot_table.index[indices.flatten()[i]]}")

    except KeyError:
        print(f"The book '{book}' is not found in the dataset.")
        similar_books = find_similar_books(book)

        if similar_books:
            print("*********************************************")
            print(f"Did you mean one of these books?")
            for i, similar_book in enumerate(similar_books, start=1):
                print(f"{i}: {similar_book}")
        else:
            print(f"No similar books found. Please try a different book name.")



def find_similar_books(book):
    similar_books = [b for b in pivot_table.index if book.lower() in b.lower()]
    return similar_books[:10] if similar_books else []


    
while True:
    # Input book name for recommendation
    print("*********************************************")
    book_name = input("Enter the Book Name (Type 'exit' to stop): ")

    if book_name.lower() == 'exit':
        print("Exiting the recommendation system.")
        print("*********************************************")
        break

    recommend(book_name)

*********************************************


Enter the Book Name (Type 'exit' to stop):  lord of the ring


The book 'lord of the ring' is not found in the dataset.
*********************************************
Did you mean one of these books?
1: Bored of the Rings: A Parody of J.R.R. Tolkien's the Lord of the Rings
2: Finding God in the Lord of the Rings
3: The Fellowship of the Ring (Lord of the Rings (Paperback))
4: The Fellowship of the Ring (The Lord of the Rings, Part 1)
5: The Hobbit : The Enchanting Prelude to The Lord of the Rings
6: The Lord of the Rings
7: The Lord of the Rings (Movie Art Cover)
8: The Return of the King (The Lord of The Rings, Part 3)
9: The Return of the King (The Lord of the Rings, Part 3)
10: The Two Towers (Lord of the Rings (Paperback))
*********************************************


Enter the Book Name (Type 'exit' to stop):  Finding God in the Lord of the Rings


Recommendations for 'Finding God in the Lord of the Rings':
1: Circus
2: Fourth Grade Rats
3: The Secret (Animorphs, No 9)
4: The Burning City
5: Criminal Justice
*********************************************


...

The book 'the lords of the ring' is not found in the dataset.
No similar books found. Please try a different book name.
*********************************************


Enter the Book Name (Type 'exit' to stop):  houses of stones


The book 'houses of stones' is not found in the dataset.
No similar books found. Please try a different book name.
*********************************************


Enter the Book Name (Type 'exit' to stop):  exit


Exiting the recommendation system.
*********************************************
